# Data preprocessing 

In [2]:
from pathlib import Path
import pandas as pd 
import numpy as np
import pickle

## Drug smiles

In [2]:
drug_smiles = pd.read_csv('E:/data/DC_drug_smiles.csv')

In [3]:
length =  []                                                                                                                                                                                                                            
for i in range(len(drug_smiles)):
    temp = len(drug_smiles.iloc[i][1])
    length.append(temp)
smiles_col = drug_smiles['smiles']
smiles_col = smiles_col.apply(lambda x: x[:200].ljust(200, '0'))
drug_smiles['smiles'] = smiles_col
##label enconding
seqs=drug_smiles['smiles'].to_list()
chars=set([char for seq in seqs for char in seq])
chars.remove('0') 
chars = ['0']+list(chars) #for zero embedding  
codes = {'mole':Mapping(chars)}  
drug_smiles['smiles']=drug_smiles['smiles'].apply(lambda x : [codes['mole'].item2idx[char] for char in x])
drug_smiles.reset_index(drop = True, inplace = True)
drug_dict = Mapping(drug_smiles['cid']).item2idx
np.save('E:/data/drug_onlysmiles.npy')
drug_smiles['cid'] = drug_smiles['cid'].apply(lambda x :drug_dict[x])
drug_smiles.to_pickle('E:/data/drug_onlysmiles.pickle')

## Expression profile 

In [4]:
ccle_fpkm = pd.read_csv('E:/data/ccle_ge_fpkm.csv',low_memory=False)
ccle_fpkm = ccle_fpkm.loc[:,ccle_fpkm.columns.isin(cell_line_name_cancertype)]
test = ccle_fpkm.loc[:,['Name']]
ccle_fpkm = pd.concat([test,ccle_fpkm_1],axis = 1)

gene_ensemble_id_split = ccle_fpkm_2.columns.tolist()
for i in range(len(gene_ensemble_id_split)):
    gene_ensemble_id_split[i] = gene_ensemble_id_split[i].split(sep = '.')[0]
gene_ensemble_id_split =pd.DataFrame(gene_ensemble_id_split)

ccle_geneid = pd.read_csv( 'E:/data/ccle_geneid.csv') 
ccle_geneid = ccle_geneid.dropna()
ccle_geneid.reset_index(drop=True, inplace=True)
ccle_geneid['NCBI gene (formerly Entrezgene) ID'] =ccle_geneid['NCBI gene (formerly Entrezgene) ID'].apply(int)
geneid_mapping = dict(zip(list(ccle_geneid['Gene stable ID'].values),list(ccle_geneid['NCBI gene (formerly Entrezgene) ID'].values)))
ccle_fpkm.columns = gene_ensemble_id_split[0]
new_ccle = ccle_fpkm.loc[:,ccle_fpkm.columns.isin(ccle_geneid['Gene stable ID'])]
new_ccle.rename(columns=lambda x:geneid_mapping[x], inplace=True)

##Z-score 
new_ccle = new_ccle.loc[:,(new_ccle.std()!=0)]
new_ccle_norm =((new_ccle-new_ccle.mean())/new_ccle.std())
new_ccle_norm.to_pickle('E:/data/cellline_feature.pickle')

## Gold standard data

In [6]:
summary = pd.read_csv('E:/data/summary_v_1_5.csv',low_memory=False)
summary = summary.dropna(axis=0,subset = ['drug_row','drug_col']) ## Remove single drug data
summary = summary.loc[summary['drug_row'] != summary['drug_col'],] ## Delete the duplicated sample for drug A and drug B
# take average of replicates experiments
summary_mean= summary.groupby(['drug_row','drug_col','cell_line_name','study_name'], as_index=False).mean()
summary_mean= summary_mean.loc[:, ['drug_row', 'drug_col', 'cell_line_name','S_sum']
dc_drug = pd.read_csv('E:/data/dc_cid.csv',dtype={"drug": str})
dc_drug = dc_drug.dropna()
dc_drug['cid'] = dc_drug['cid'].apply(int)
cid_mapping = dict(zip(list(dc_drug['drug'].values),dc_drug['cid'].values))
df = summary_mean
df = df[df['drug_row'].isin(list(dc_drug['drug'].values))]
df = df[df['drug_col'].isin(list(dc_drug['drug'].values))]
df['drug_row'] = df['drug_row'].apply(lambda x: cid_mapping[x])
df['drug_col'] = df['drug_col'].apply(lambda x: cid_mapping[x])
df.reset_index(drop=True, inplace=True)
# Delete the replicates information about drug A-B and B-A
df['drug_row'] = df['drug_row'].apply(lambda x: str(x))
df['drug_col'] = df['drug_col'].apply(lambda x: str(x))
df['sorted_drug'] = df[['drug_row', 'drug_col']].apply(lambda x: '//'.join(sorted(x, key=lambda y: y[0])), axis=1)
df =df.drop_duplicates(subset = ['sorted_drug','cell_line_name'])
df.reset_index(drop=True, inplace=True)
df[['drug_A', 'drug_B']] = df['sorted_drug'].str.split('//', expand=True)
df['drug_A'] = df['drug_A'].apply(lambda x: int(x))
df['drug_B'] = df['drug_B'].apply(lambda x: int(x))
df.reset_index(drop=True, inplace=True)
df = df[['drug_A','drug_B','cell_line_name','S_sum','CSS']]
                               
##Matching expression profile information                            
cl = df
dc_depmep=pd.read_pickle('E:/data/dc_depmep.pickle')
cl1_mapping = dict(zip(list(dc_depmep['cell_line_name'].values),list(dc_depmep['CCLE_Name'].values)))
cl1 = cl[cl['cell_line_name'].isin(dc_depmep['cell_line_name'])]
cl1['cell_line_name']=cl1['cell_line_name'].apply(lambda x: cl1_mapping[x])
cl2_mapping = dict(zip(list(dc_depmep['stripped_cell_line_name'].values),list(dc_depmep['CCLE_Name'].values)))
cl2= cl[cl['cell_line_name'].isin(dc_depmep['stripped_cell_line_name'])]
cl2['cell_line_name']=cl2['cell_line_name'].apply(lambda x: cl2_mapping[x])
cl_all = pd.concat([cl1,cl2],axis = 0)
cl_all =cl_all.drop_duplicates(subset = ['drug_A','drug_B','cell_line_name'],keep = 'first')
cell_dict = np.load('E:/data/cell_line/dict_cell.npy', allow_pickle='TRUE').item()
cl_all =cl_all[cl_all['cell_line_name'].isin(cell_dict.keys())]
cl_all['cell_line_name'] = cl_all['cell_line_name'].apply(lambda x: cell_dict[x])
cl_all.to_pickle('E:/data/preprocessed_cl_label_data.pickle')                    

In [7]:
df = pd.read_pickle('E:/data/preprocessed_cl_label_data.pickle') 
drug_dict = np.load('/E:/data/drug_onlysmiles.npy', allow_pickle='TRUE').item()
df = df[df['drug_A'].isin(drug_dict.keys())]
df = df[df['drug_B'].isin(drug_dict.keys())]
df['drug_A'] = df['drug_A'].apply(lambda x: drug_dict[x])
df['drug_B'] = df['drug_B'].apply(lambda x: drug_dict[x])
df.reset_index(drop = True,inplace = True)
label_list = []
for i  in range(len(df)):
    css = df.iloc[i][3]
    s = df.iloc[i][4]
    if (css >= 10 ) & (s > 5):
        label  = 1
    else:
        label  = 0
    label_list.append(label)

c={"label":label_list} 
data=pd.DataFrame(c)
cl_label = pd.concat([df,data],axis = 1)
cl_label.to_pickle('E:/data/cl_label_data.pickle')